In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)

In [ ]:
def convert_unit(s):
  # s = ''.join(s.split('억'))
  s = s.replace('억', '').replace('개', '').replace(',', '')
  s = s.replace('만', '0000')
  return f'{int(s):,d}'

In [ ]:
trs = driver.find_elements_by_css_selector('.aos-init')

In [ ]:
rs = []
for inedx, tr in enumerate(trs):
  category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
  subscriber = tr.find_element_by_css_selector('.subscriber_cnt').text
  name = tr.find_element_by_css_selector('.subject a').text.strip()
  view = tr.find_element_by_css_selector('.view_cnt').text
  video = tr.find_element_by_css_selector('.video_cnt').text
  rs.append([category, name, subscriber, view, video])

In [ ]:
df = pd.DataFrame(rs, columns=['카테고리', '채널', '구독자수', '조회수', '동영상개수'])
df

,카테고리,채널,구독자수,조회수,동영상개수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371개
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654개
2,,,,,
3,,,,,
4,,,,,
...,...,...,...,...,...
95,,,,,
96,,,,,
97,,,,,
98,,,,,


In [ ]:
results = []

for page in range(1, 11):
  url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+ str(page)
  driver.get(url)
  time.sleep(3)

  window_len = len(driver.window_handles)
  if window_len == 2:
    driver.switch_to_window(driver.window_handles[1])
    driver.close()
  driver.switch_to_window(driver.window_handles[0])
  time.sleep(1)

  trs = driver.find_elements_by_css_selector('.aos-init')

  for tr in trs:
    category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
    # category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = tr.find_element_by_css_selector('.subject a').text.strip()
    # name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
    # subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
    # view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
    # video = convert_unit(channel.select_one('.video_cnt').text[:-1])
    results.append([category, name, subscriber, view, video])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use driver.switch_to.window instead
  if sys.path[0] == '':


ValueError: ignored

In [ ]:
df = df = pd.DataFrame(results, columns=['카테고리', '채널', '구독자수', '조회수', '동영상개수'])
df.head()